In [1]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS

Before proceeding, add a logger and set logging level to `DEBUG` so we can get status message from the various tasks.

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

Next we initialize two high level classes to command the ATTCS and LATISS.

In [3]:
attcs = ATTCS()

Using selector: EpollSelector


In [4]:
await attcs.start_task

Read historical data in 0.01 sec
Read historical data in 0.55 sec
Read historical data in 0.69 sec
Read historical data in 1.14 sec
Read historical data in 2.04 sec
Read historical data in 2.57 sec
Read historical data in 3.36 sec
RemoteEvent(ATDome, 0, logMessage) falling behind; read 17 messages


[None, None, None, None, None, None, None]

In [ ]:
await asyncio.sleep(1)
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED)

In [ ]:
# attcs.check.athexapod = False
await asyncio.sleep(5)

In [ ]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "current",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

In [ ]:
await attcs.atdometrajectory.cmd_enable.start()

In [ ]:
await attcs.atdome.cmd_moveAzimuth.set_start(azimuth=90)

In [ ]:
await attcs.atmcs.cmd_stopTracking.start()

In [ ]:
await attcs.startup()

In [ ]:
await attcs.atpneumatics.cmd_closeInstrumentAirValve.start()

In [ ]:
await attcs.atpneumatics.cmd_closeMasterAirSupply.start()

In [ ]:
await attcs.ataos.cmd_enableCorrection.set_start(m1=False, hexapod=True, timeout=30)

In [ ]:
await attcs.ataos.cmd_disableCorrection.set_start(m1=False, hexapod=True, timeout=30)

In [ ]:
await attcs.atpneumatics.cmd_openM1Cover.start()

In [ ]:
await attcs.atdome.cmd_homeAzimuth.start()

Now we can use LATISS high level method `take_bias` to take a series of bias images.

In [ ]:
await attcs.startup()

In [ ]:
attcs.check.atdome = False

In [ ]:
await attcs.atdome.cmd_closeShutter.start()

Select instrument port. This is not done at the startup.

In [ ]:
time = await attcs.atptg.tel_timeAndDate.aget()

In [ ]:
time.lst

In [ ]:
await attcs.slew_icrs(ra=time.lst, dec=-50., slew_timeout=120)

In [ ]:
await salobj.set_summary_state(attcs.atptg, salobj.State.STANDBY)

In [ ]:
await asyncio.sleep(5)
await asyncio.gather(salobj.set_summary_state(attcs.atmcs, salobj.State.STANDBY, timeout=120),
                     salobj.set_summary_state(attcs.atptg, salobj.State.STANDBY),
                     salobj.set_summary_state(attcs.ataos, salobj.State.STANDBY),
                     salobj.set_summary_state(attcs.atpneumatics, salobj.State.STANDBY),
                     salobj.set_summary_state(attcs.athexapod, salobj.State.STANDBY),
                     salobj.set_summary_state(attcs.atdome, salobj.State.STANDBY),
                     salobj.set_summary_state(attcs.atdometrajectory, salobj.State.STANDBY))